<a href="https://colab.research.google.com/github/SeniyaSultan/bert-sentiment-analysis-eco-products/blob/main/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Install required libraries (uncomment if not already installed)
!pip install transformers scikit-learn

In [ ]:
# STEP 2: Import libraries
import random
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

In [ ]:
#  STEP 3: Generate synthetic eco-tweets
positive_data = [f"This eco-friendly product is amazing! #{i}" for i in range(500)]
negative_data = [f"This green product is a total scam. #{i}" for i in range(500)]

df_pos = pd.DataFrame({'text': positive_data, 'label': 1})
df_neg = pd.DataFrame({'text': negative_data, 'label': 0})
df = pd.concat([df_pos, df_neg]).sample(frac=1).reset_index(drop=True)

In [ ]:
#  STEP 4: Split train/val
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
# STEP 5: Tokenization with BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#  STEP 6: Dataset class
class EcoTweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = EcoTweetDataset(train_encodings, train_labels)
val_dataset = EcoTweetDataset(val_encodings, val_labels)

In [ ]:

# STEP 7: Load BERT
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#  STEP 8: Training setup
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,                # Can be 1 for fast test
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"                  # no wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
# STEP 9: Train the model
trainer.train()

Step,Training Loss
10,0.312200
20,0.029500
30,0.005000
40,0.001800
50,0.001000
60,0.000800
70,0.000600
80,0.000500
90,0.000500
100,0.000400


TrainOutput(global_step=150, training_loss=0.023611470946731668, metrics={'train_runtime': 574.0154, 'train_samples_per_second': 4.181, 'train_steps_per_second': 0.261, 'total_flos': 17266663008000.0, 'train_loss': 0.023611470946731668, 'epoch': 3.0})

In [ ]:
test_texts = [
    "This degradable plastic breaks down in a week!",
    "Not eco-friendly at all. Waste of money."
]

test_encodings = tokenizer(test_texts, return_tensors="pt", truncation=True, padding=True)
outputs = model(**test_encodings)
predictions = torch.argmax(outputs.logits, dim=1)

for text, pred in zip(test_texts, predictions):
    print(f"{text} → {'Positive' if pred == 1 else 'Negative'}")


This degradable plastic breaks down in a week! → Positive
Not eco-friendly at all. Waste of money. → Positive
